In [3]:
import sys
import os
import pandas as pd

sys.path.append(os.path.abspath("../../ml/"))

from dev_wtp_io_utils import *

In [4]:
# nonbids_data_path = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/"
# ml_data_folderpath = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml"

sys.path.append(os.path.abspath("../../ml/"))
from apply_loocv_and_save import load_and_preprocess
from dev_utils import read_yaml_for_host
import warnings


config_data = read_yaml_for_host("sst_config.yml")



In [5]:
nonbids_data_path = config_data['nonbids_data_path']
ml_data_folderpath = nonbids_data_path + "fMRI/ml"


In [6]:
#test_train_df_raw = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20210601T183243.csv")
#test_train_df_raw = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20211027T173724.csv")
test_train_df_raw = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20220818T144138.csv")

In [7]:
include_exclude_list = pd.read_csv("../nsc_subject_exclusions.csv")

In [8]:
test_train_df_raw = test_train_df_raw.merge(include_exclude_list[include_exclude_list.Task=='SST'],left_on='sub_label',right_on='SubjectId',how='left')
test_train_df_raw.loc[test_train_df_raw.Include.isna(),'Include'] = True
test_train_df = test_train_df_raw[test_train_df_raw.Include==True]

In [9]:
exclude_subjects = ['DEV061','DEV185','DEV187','DEV189','DEV190','DEV192','DEV198','DEV203','DEV220','DEV221']
#2022-01-06 added DEV157 because of an error

In [10]:
train_subjs = test_train_df.loc[test_train_df.SplitGroup=='Train','sub_label'].tolist()#only get the train subjects; ignore those previously marked hold-out

In [11]:
def get_Brain_Data_for_sub(sub_label,behavdesign,betaseries_path='/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries/'):
    return(dev_wtp_io_utils.get_Brain_Data_betas_for_sub(sub_label,behavdesign,betaseries_path))

In [12]:
from dev_wtp_io_utils import import_sst_betaseries_w1_subjs_to_pkl

In [13]:
all_sst_events= pd.read_csv(ml_data_folderpath +"/SST/" + "all_sst_events.csv")
#cut out other waves
sst_1_behavdesign_1 = all_sst_events[(all_sst_events.wave=='wave1')].reset_index(drop=True)
#label the ITI as such
sst_1_behavdesign_1.loc[(all_sst_events.trial_type.isna()),'trial_type']='ITI'
#ensure it is correctly ordered
sst_1_behavdesign_clean = sst_1_behavdesign_1.sort_values(['subject','onset']).reset_index(drop=True)
#now tell it the name of the corresponding beta. this is easy; they are all consecutive
sst_1_behavdesign_clean['beta'] = 'beta_' + (sst_1_behavdesign_clean.groupby(['subject','wave']).cumcount()+1).astype(str).apply(lambda x: x.zfill(4)) + '.nii'

In [14]:
#I want to import this other module; we don't actually use it but it's a good reference
#for future programmers to see
#it's important we keep the logic consistent for the two main conditions
sys.path.append(os.path.abspath("../../fx/multiconds/SST/"))
from multiconds import create_posterror_masks_from_masks

#sst_1_behavdesign_clean['prev_trial_type'] = sst_1_behavdesign_clean['trial_type']
sst_1_behavdesign_clean['prev_trial_type'] = (
    sst_1_behavdesign_clean
    .loc[sst_1_behavdesign_clean['trial_type']!='ITI']
    .groupby(['subject','wave'])
    .shift(1)
    ['trial_type']
)#this works because during assignment, items will be assigned to the line corresponding to their index
#very important you don't reset index during this operation.
is_post_stop_correct_go = (
    (sst_1_behavdesign_clean['trial_type']=='correct-go') & 
    (sst_1_behavdesign_clean['prev_trial_type'].isin(['failed-stop','correct-stop']))
)

sst_1_behavdesign_clean.loc[is_post_stop_correct_go,'post_stop_correct_go_subtype'] = 'Other'
sst_1_behavdesign_clean.loc[is_post_stop_correct_go & (sst_1_behavdesign_clean['prev_trial_type']=='failed-stop'),'post_stop_correct_go_subtype'] = 'post-error'
sst_1_behavdesign_clean.loc[is_post_stop_correct_go & (sst_1_behavdesign_clean['prev_trial_type']=='correct-stop'),'post_stop_correct_go_subtype'] = 'post-correct'


In [15]:
sst_1_behavdesign_clean.groupby(['trial_type','post_stop_correct_go_subtype']).size().reset_index()

,trial_type,post_stop_correct_go_subtype,0
0,correct-go,post-correct,1879
1,correct-go,post-error,2145


In [16]:
sst_1_behavdesign_clean['trial_type'].value_counts()

ITI             26112
correct-go      18750
failed-stop      3434
correct-stop     3094
failed-go         834
Name: trial_type, dtype: int64

In [17]:
behavdesign = sst_1_behavdesign_clean
sst_1_behavdesign_clean_dev005 = behavdesign[[s in 'DEV005' for s in behavdesign.subject]]

## Just subjects in the conditions of interest: CorrectStop vs. CorrectGo

In [18]:
sst_1_behavdesign_clean.trial_type.value_counts()

ITI             26112
correct-go      18750
failed-stop      3434
correct-stop     3094
failed-go         834
Name: trial_type, dtype: int64

In [19]:
sst_1_behavdesign_correct_conditions=sst_1_behavdesign_clean[sst_1_behavdesign_clean.trial_type.isin(['correct-go','correct-stop'])]

In [20]:
sst_1_behavdesign_correct_conditions.trial_type.value_counts()

correct-go      18750
correct-stop     3094
Name: trial_type, dtype: int64

In [21]:
#we have data missing fro mthese subjects below. not enstirely sure yet; see 
#https://docs.google.com/document/d/1EB0ACA6qhMkDEgv-zqNTepPCe_x41fRQHaeqT47vazM/edit#heading=h.60rb13rlxkrq


train_subjs_selected = [ts for ts in train_subjs if (ts not in exclude_subjects)]

In [22]:
sst_1_behavdesign_correct_conditions[sst_1_behavdesign_correct_conditions.subject=='DEV157']

,onset,duration,trial_type,subject,wave,beta,prev_trial_type,post_stop_correct_go_subtype
37376,0.00000,0.87197,correct-go,DEV157,wave1,beta_0001.nii,NaN,NaN
37380,6.38890,0.08329,correct-go,DEV157,wave1,beta_0005.nii,failed-stop,post-error
37384,15.65280,0.59839,correct-go,DEV157,wave1,beta_0009.nii,failed-stop,post-error
37386,20.53475,0.46574,correct-go,DEV157,wave1,beta_0011.nii,correct-go,NaN
37388,23.35212,0.34208,correct-go,DEV157,wave1,beta_0013.nii,correct-go,NaN
...,...,...,...,...,...,...,...,...
37618,400.61111,0.43916,correct-go,DEV157,wave1,beta_0243.nii,correct-go,NaN
37620,404.11667,0.72024,correct-go,DEV157,wave1,beta_0245.nii,correct-go,NaN
37622,407.24723,0.43989,correct-go,DEV157,wave1,beta_0247.nii,correct-go,NaN
37626,412.44516,0.00170,correct-go,DEV157,wave1,beta_0251.nii,failed-stop,post-error


### Now mask those subjects

In [23]:
# get the PFC mask
mask_pfc = nib.load(ml_data_folderpath + '/prefrontal_cortex.nii.gz')

In [24]:
print ("running")
print('really')
import_sst_betaseries_w1_subjs_to_pkl(
    train_subjs[0:2],'betaseries',
    behavioral_design=sst_1_behavdesign_correct_conditions,
    out_folder = ml_data_folderpath + "/SST/",
    mask = mask_pfc,
    out_file_suffix = '_correct_cond',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/")

running
really
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries/
DEV005
........................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
...using a combined mask of the first image and the passed in maskDuring mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
         onset  duration    trial_type subject   wave           beta  \
256    0.00000   2.25834  correct-stop  DEV005  wave1  beta_0001.nii   
258    2.75834   0.40082    correct-go  DEV005  wave1  beta_0003.nii   
260    5.51390   0.66191    correct-go  DEV005  wave1  beta_0005.nii   
264   12.52780   0.51712    correct-go  DEV005  wave1  beta_0009.nii   
266   15.90975   0.39906    correct-go  DEV

In [26]:
print ("running")
print('really')
import_sst_betaseries_w1_subjs_to_pkl(
    train_subjs[0:30],'betaseries',
    behavioral_design=sst_1_behavdesign_correct_conditions,
    out_folder = ml_data_folderpath + "/SST/",
    mask = mask_pfc,
    out_file_suffix = '_correct_cond_pfc',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/")

running
really
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries/
DEV005
........................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
...using a combined mask of the first image and the passed in maskDuring mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
         onset  duration    trial_type subject   wave           beta  \
256    0.00000   2.25834  correct-stop  DEV005  wave1  beta_0001.nii   
258    2.75834   0.40082    correct-go  DEV005  wave1  beta_0003.nii   
260    5.51390   0.66191    correct-go  DEV005  wave1  beta_0005.nii   
264   12.52780   0.51712    correct-go  DEV005  wave1  beta_0009.nii   
266   15.90975   0.39906    correct-go  DEV

...............................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0003.nii
...using a combined mask of the first image and the passed in maskDuring mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
          onset  duration  trial_type subject   wave           beta  \
1794    2.75834   0.64209  correct-go  DEV011  wave1  beta_0003.nii   
1796    5.51390   0.56936  correct-go  DEV011  wave1  beta_0005.nii   
1798    9.02224   0.46994  correct-go  DEV011  wave1  beta_0007.nii   
1800   12.52780   0.40205  correct-go  DEV011  wave1  beta_0009.nii   
1802   15.90975   0.44627  correct-go  DEV011  wave1  beta_0011.nii   
...         ...       ...         ...     ...    ...       

........................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0007.nii
...using a combined mask of the first image and the passed in maskDuring mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
          onset  duration    trial_type subject   wave           beta  \
2822    9.02224   0.68000    correct-go  DEV015  wave1  beta_0007.nii   
2824   12.52780   0.84117    correct-go  DEV015  wave1  beta_0009.nii   
2826   15.90975   0.47052    correct-go  DEV015  wave1  beta_0011.nii   
2828   18.35212   0.75253    correct-go  DEV015  wave1  beta_0013.nii   
2832   24.42644   0.55314    correct-go  DEV015  wave1  beta_0017.nii   
...         ...       ...           ...     ...    ...

....................................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
...using a combined mask of the first image and the passed in maskDuring mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
          onset  duration    trial_type subject   wave           beta  \
3840    0.00000   0.77984    correct-go  DEV019  wave1  beta_0001.nii   
3844    6.38890   0.80940    correct-go  DEV019  wave1  beta_0005.nii   
3848   15.65280   0.72471    correct-go  DEV019  wave1  beta_0009.nii   
3850   20.53475   0.56789    correct-go  DEV019  wave1  beta_0011.nii   
3852   23.35212   0.89164    correct-go  DEV019  wave1  beta_0013.nii   
...         ...       ...           ...   

...............................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0003.nii
...using a combined mask of the first image and the passed in maskDuring mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
          onset  duration    trial_type subject   wave           beta  \
4866    2.75834   0.59212    correct-go  DEV023  wave1  beta_0003.nii   
4868    5.51390   0.44724    correct-go  DEV023  wave1  beta_0005.nii   
4870    9.02224   0.43165    correct-go  DEV023  wave1  beta_0007.nii   
4872   12.52780   0.47762    correct-go  DEV023  wave1  beta_0009.nii   
4874   15.90975   0.39854    correct-go  DEV023  wave1  beta_0011.nii   
...         ...       ...           ...     ...

.....................................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0003.nii
...using a combined mask of the first image and the passed in maskDuring mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
          onset  duration    trial_type subject   wave           beta  \
5890    3.00834   2.00556  correct-stop  DEV027  wave1  beta_0003.nii   
5892    6.38890   0.88834    correct-go  DEV027  wave1  beta_0005.nii   
5894   11.77224   2.00556  correct-stop  DEV027  wave1  beta_0007.nii   
5896   15.65280   0.69325    correct-go  DEV027  wave1  beta_0009.nii   
5898   20.53475   0.71644    correct-go  DEV027  wave1  beta_0011.nii   
...         ...       ...           ...  

............................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
...using a combined mask of the first image and the passed in maskDuring mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
          onset  duration    trial_type subject   wave           beta  \
7680    0.00000   0.89262    correct-go  DEV034  wave1  beta_0001.nii   
7682    3.00834   2.00556  correct-stop  DEV034  wave1  beta_0003.nii   
7684    6.38890   0.46958    correct-go  DEV034  wave1  beta_0005.nii   
7688   15.65280   0.72179    correct-go  DEV034  wave1  beta_0009.nii   
7690   20.53475   0.52387    correct-go  DEV034  wave1  beta_0011.nii   
...         ...       ...           ...     ...   

...................................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
...using a combined mask of the first image and the passed in maskDuring mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
          onset  duration    trial_type subject   wave           beta  \
9216    0.00000   2.25834  correct-stop  DEV040  wave1  beta_0001.nii   
9218    2.75834   0.49211    correct-go  DEV040  wave1  beta_0003.nii   
9220    5.51390   0.65179    correct-go  DEV040  wave1  beta_0005.nii   
9222    9.02224   0.56454    correct-go  DEV040  wave1  beta_0007.nii   
9224   12.52780   0.48032    correct-go  DEV040  wave1  beta_0009.nii   
...         ...       ...           ...    

In [28]:
print ("running")
print('really')
import_sst_betaseries_w1_subjs_to_pkl(
    train_subjs,'betaseries',
    behavioral_design=sst_1_behavdesign_correct_conditions,
    out_folder = ml_data_folderpath + "/SST/",
    mask = mask_pfc,
    out_file_suffix = '_correct_cond_pfc',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/")

running
really
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries/
DEV005
........................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
...using a combined mask of the first image and the passed in maskDuring mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
         onset  duration    trial_type subject   wave           beta  \
256    0.00000   2.25834  correct-stop  DEV005  wave1  beta_0001.nii   
258    2.75834   0.40082    correct-go  DEV005  wave1  beta_0003.nii   
260    5.51390   0.66191    correct-go  DEV005  wave1  beta_0005.nii   
264   12.52780   0.51712    correct-go  DEV005  wave1  beta_0009.nii   
266   15.90975   0.39906    correct-go  DEV

...............................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0003.nii
...using a combined mask of the first image and the passed in maskDuring mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
          onset  duration  trial_type subject   wave           beta  \
1794    2.75834   0.64209  correct-go  DEV011  wave1  beta_0003.nii   
1796    5.51390   0.56936  correct-go  DEV011  wave1  beta_0005.nii   
1798    9.02224   0.46994  correct-go  DEV011  wave1  beta_0007.nii   
1800   12.52780   0.40205  correct-go  DEV011  wave1  beta_0009.nii   
1802   15.90975   0.44627  correct-go  DEV011  wave1  beta_0011.nii   
...         ...       ...         ...     ...    ...       

........................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0007.nii
...using a combined mask of the first image and the passed in maskDuring mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
          onset  duration    trial_type subject   wave           beta  \
2822    9.02224   0.68000    correct-go  DEV015  wave1  beta_0007.nii   
2824   12.52780   0.84117    correct-go  DEV015  wave1  beta_0009.nii   
2826   15.90975   0.47052    correct-go  DEV015  wave1  beta_0011.nii   
2828   18.35212   0.75253    correct-go  DEV015  wave1  beta_0013.nii   
2832   24.42644   0.55314    correct-go  DEV015  wave1  beta_0017.nii   
...         ...       ...           ...     ...    ...

....................................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
...using a combined mask of the first image and the passed in maskDuring mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
          onset  duration    trial_type subject   wave           beta  \
3840    0.00000   0.77984    correct-go  DEV019  wave1  beta_0001.nii   
3844    6.38890   0.80940    correct-go  DEV019  wave1  beta_0005.nii   
3848   15.65280   0.72471    correct-go  DEV019  wave1  beta_0009.nii   
3850   20.53475   0.56789    correct-go  DEV019  wave1  beta_0011.nii   
3852   23.35212   0.89164    correct-go  DEV019  wave1  beta_0013.nii   
...         ...       ...           ...   

...............................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0003.nii
...using a combined mask of the first image and the passed in maskDuring mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
          onset  duration    trial_type subject   wave           beta  \
4866    2.75834   0.59212    correct-go  DEV023  wave1  beta_0003.nii   
4868    5.51390   0.44724    correct-go  DEV023  wave1  beta_0005.nii   
4870    9.02224   0.43165    correct-go  DEV023  wave1  beta_0007.nii   
4872   12.52780   0.47762    correct-go  DEV023  wave1  beta_0009.nii   
4874   15.90975   0.39854    correct-go  DEV023  wave1  beta_0011.nii   
...         ...       ...           ...     ...

...

..........................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
...using a combined mask of the first image and the passed in maskDuring mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
           onset  duration    trial_type subject   wave           beta  \
25088    0.00000   2.25834  correct-stop  DEV105  wave1  beta_0001.nii   
25090    2.75834   0.24692    correct-go  DEV105  wave1  beta_0003.nii   
25094    9.02224   0.73690    correct-go  DEV105  wave1  beta_0007.nii   
25096   12.52780   0.90126    correct-go  DEV105  wave1  beta_0009.nii   
25098   15.90975   0.68569    correct-go  DEV105  wave1  beta_0011.nii   
...          ...       ...           ...     .

................................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0003.nii
...using a combined mask of the first image and the passed in maskDuring mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
           onset  duration    trial_type subject   wave           beta  \
34818    2.75834   0.61486    correct-go  DEV145  wave1  beta_0003.nii   
34820    5.51390   0.37433    correct-go  DEV145  wave1  beta_0005.nii   
34822    9.02224   0.57750    correct-go  DEV145  wave1  beta_0007.nii   
34824   12.52780   0.49848    correct-go  DEV145  wave1  beta_0009.nii   
34826   15.90975   0.46570    correct-go  DEV145  wave1  beta_0011.nii   
...          ...       ...           ...

......................................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0005.nii
...using a combined mask of the first image and the passed in maskDuring mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
           onset  duration    trial_type subject   wave           beta  \
38660    6.38890   0.91166    correct-go  DEV164  wave1  beta_0005.nii   
38662   11.77224   2.00556  correct-stop  DEV164  wave1  beta_0007.nii   
38664   15.65280   0.67040    correct-go  DEV164  wave1  beta_0009.nii   
38666   20.53475   0.57541    correct-go  DEV164  wave1  beta_0011.nii   
38668   23.35212   0.58260    correct-go  DEV164  wave1  beta_0013.nii   
...          ...       ...        

..................................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
...using a combined mask of the first image and the passed in maskDuring mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
           onset  duration    trial_type subject   wave           beta  \
41472    0.00000   0.79588    correct-go  DEV177  wave1  beta_0001.nii   
41476    6.38890   0.84028    correct-go  DEV177  wave1  beta_0005.nii   
41480   15.65280   0.52386    correct-go  DEV177  wave1  beta_0009.nii   
41482   20.53475   0.48580    correct-go  DEV177  wave1  beta_0011.nii   
41484   23.35212   0.39875    correct-go  DEV177  wave1  beta_0013.nii   
...          ...       ...           .

Exception: beta beta_0005.nii does not exist at /gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries/sub-DEV185/beta_0005.nii

In [29]:
print ("running")
print('really')
import_sst_betaseries_w1_subjs_to_pkl(
    train_subjs,'betaseries',
    behavioral_design=sst_1_behavdesign_correct_conditions,
    out_folder = ml_data_folderpath + "/SST/",
    mask = mask_pfc,
    out_file_suffix = '_correct_cond_pfc',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/")

running
really
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries/
DEV005
........................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
...using a combined mask of the first image and the passed in maskDuring mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
         onset  duration    trial_type subject   wave           beta  \
256    0.00000   2.25834  correct-stop  DEV005  wave1  beta_0001.nii   
258    2.75834   0.40082    correct-go  DEV005  wave1  beta_0003.nii   
260    5.51390   0.66191    correct-go  DEV005  wave1  beta_0005.nii   
264   12.52780   0.51712    correct-go  DEV005  wave1  beta_0009.nii   
266   15.90975   0.39906    correct-go  DEV

...............................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0003.nii
...using a combined mask of the first image and the passed in maskDuring mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
          onset  duration  trial_type subject   wave           beta  \
1794    2.75834   0.64209  correct-go  DEV011  wave1  beta_0003.nii   
1796    5.51390   0.56936  correct-go  DEV011  wave1  beta_0005.nii   
1798    9.02224   0.46994  correct-go  DEV011  wave1  beta_0007.nii   
1800   12.52780   0.40205  correct-go  DEV011  wave1  beta_0009.nii   
1802   15.90975   0.44627  correct-go  DEV011  wave1  beta_0011.nii   
...         ...       ...         ...     ...    ...       

........................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0007.nii
...using a combined mask of the first image and the passed in maskDuring mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
          onset  duration    trial_type subject   wave           beta  \
2822    9.02224   0.68000    correct-go  DEV015  wave1  beta_0007.nii   
2824   12.52780   0.84117    correct-go  DEV015  wave1  beta_0009.nii   
2826   15.90975   0.47052    correct-go  DEV015  wave1  beta_0011.nii   
2828   18.35212   0.75253    correct-go  DEV015  wave1  beta_0013.nii   
2832   24.42644   0.55314    correct-go  DEV015  wave1  beta_0017.nii   
...         ...       ...           ...     ...    ...

....................................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
...using a combined mask of the first image and the passed in maskDuring mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
          onset  duration    trial_type subject   wave           beta  \
3840    0.00000   0.77984    correct-go  DEV019  wave1  beta_0001.nii   
3844    6.38890   0.80940    correct-go  DEV019  wave1  beta_0005.nii   
3848   15.65280   0.72471    correct-go  DEV019  wave1  beta_0009.nii   
3850   20.53475   0.56789    correct-go  DEV019  wave1  beta_0011.nii   
3852   23.35212   0.89164    correct-go  DEV019  wave1  beta_0013.nii   
...         ...       ...           ...   

...............................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0003.nii
...using a combined mask of the first image and the passed in maskDuring mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
          onset  duration    trial_type subject   wave           beta  \
4866    2.75834   0.59212    correct-go  DEV023  wave1  beta_0003.nii   
4868    5.51390   0.44724    correct-go  DEV023  wave1  beta_0005.nii   
4870    9.02224   0.43165    correct-go  DEV023  wave1  beta_0007.nii   
4872   12.52780   0.47762    correct-go  DEV023  wave1  beta_0009.nii   
4874   15.90975   0.39854    correct-go  DEV023  wave1  beta_0011.nii   
...         ...       ...           ...     ...

.....................................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0003.nii
...using a combined mask of the first image and the passed in maskDuring mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
          onset  duration    trial_type subject   wave           beta  \
5890    3.00834   2.00556  correct-stop  DEV027  wave1  beta_0003.nii   
5892    6.38890   0.88834    correct-go  DEV027  wave1  beta_0005.nii   
5894   11.77224   2.00556  correct-stop  DEV027  wave1  beta_0007.nii   
5896   15.65280   0.69325    correct-go  DEV027  wave1  beta_0009.nii   
5898   20.53475   0.71644    correct-go  DEV027  wave1  beta_0011.nii   
...         ...       ...           ...  

............................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
...using a combined mask of the first image and the passed in maskDuring mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
          onset  duration    trial_type subject   wave           beta  \
7680    0.00000   0.89262    correct-go  DEV034  wave1  beta_0001.nii   
7682    3.00834   2.00556  correct-stop  DEV034  wave1  beta_0003.nii   
7684    6.38890   0.46958    correct-go  DEV034  wave1  beta_0005.nii   
7688   15.65280   0.72179    correct-go  DEV034  wave1  beta_0009.nii   
7690   20.53475   0.52387    correct-go  DEV034  wave1  beta_0011.nii   
...         ...       ...           ...     ...   

...................................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
...using a combined mask of the first image and the passed in maskDuring mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
          onset  duration    trial_type subject   wave           beta  \
9216    0.00000   2.25834  correct-stop  DEV040  wave1  beta_0001.nii   
9218    2.75834   0.49211    correct-go  DEV040  wave1  beta_0003.nii   
9220    5.51390   0.65179    correct-go  DEV040  wave1  beta_0005.nii   
9222    9.02224   0.56454    correct-go  DEV040  wave1  beta_0007.nii   
9224   12.52780   0.48032    correct-go  DEV040  wave1  beta_0009.nii   
...         ...       ...           ...    

KeyboardInterrupt: 